## Delta-Learning : ML-based technique  
With an attempt to adopt the so-called 'Delta-Learning' technique, this jupyter notebook aims to read the projections at 
atomic level to different eigenstates of the system using the input and output files from PBE calculations. The eventual goal is to predict the eigenvalues of HSE06 quality, using those information.
In a nutshell, get predictions that has accuracy of higher level calculations using results of the lower level calculations 
bypassing the need to actually do the computationally demanding high-level calculations. As usual we start by importing the 
necessary modules. Note: the following part of the code was written by @Jacob Clary

In [ ]:
from __future__ import annotations

#common modules
import os, sys
import numpy as np
import math
import scipy.ndimage as scind
import json
from functools import wraps
from typing import Type
from typing import Optional
import warnings

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib.lines import Line2D
import matplotlib.colors as colors
from matplotlib import cm
from matplotlib.colors import ListedColormap
import seaborn as sns

Following block of codes contain the necessary functions. Check for the text commented sections for more details.

In [ ]:
def readjson(filename):
    '''Read in filename as a json and return contents as a dict'''
    with open(filename) as json_file:
        data = json.load(json_file)
    return data

def writejson(filename, dictdata):
    '''Write dictdata (a dict) to a json file with name filename'''
    with open(filename,'w') as outfile:
        json.dump(dictdata, outfile)

def doesfileexist(func):
    '''Check if file exists (and continue normally) or raise an exception if it does not'''
    @wraps(func)
    def wrapper(filename):
        if not os.path.isfile(filename):
            raise OSError('\'' + filename + '\' file doesn\'t exist!')
        return func(filename)
    return wrapper

def stringifyarray(array, delimter = ' ', minspaces = 1, align = 'right'):
    '''
    Takes an inputted array and merges into it 1 string

    Parameters
    ----------
    array : numpy array of str or list of lists of str
        data to join
    delimiter : str, optional
        character to join list entries with (default is ' ')
    minspaces : int, optional
        minimum number of spaces allocated to each entry (default is 1)
    align : str, optional, choices: ['right', 'left', 'center']
        where to align string within it's allocated spaces (default is right)

    Returns
    -------
    joined data as a str
    '''
    if align == 'left':
        alignchar = '<'
    elif align == 'center':
        alignchar = '^'
    else:
        alignchar = '>'
    if minspaces <= 1:
        minspaces = 1
    return delimter.join(['{:{}{}}'.format(x, alignchar, minspaces) for x in array])

def stringifyfloatarray(array, delimiter = ' ', prec = 16):
    '''
    Takes an inputted array and merges into it 1 string

    Parameters
    ----------
    array : numpy array of floats or list of lists of floats
        data to join
    delimiter : str, optional
        character to join list entries with (default is ' ')
    prec : int, optional
        minimum number of places after the decimal to display (default is 16)

    Returns
    -------
    joined data as a str
    '''
    return delimiter.join(['{: .{}f}'.format(x, prec) for x in array])

class textfile():
    '''
    A class to read in a generic text file into a list of strings for later processing

    ...

    Attributes
    ----------
    filename : str
        name of file to read
    data : list of str
        file data
    Nlines: int
        number of lines in file

    Methods
    -------
    readtext(filename):
        Creates a class instance by reading in filename to populate this object with data

    readfile(filename):
        Reads a file into a list of str

    writefile(filename = 'mytestfile.txt'):
        Writes a list of str into a file

    findkey(keyinput, startline = 0, endline = None):
        Searchs for keyinput in textfile object and returns which lines contain keyinput

    extractdata(line, startindex, endindex = np.nan, splitchar = None):
        Returns specific piece of data within a line of text determined by the indices and splitchar

    discarddata():
        Deletes the data in the textfile data attribute to save memory
    '''

    def __init__(self, filename = None, initdata = None):
        '''
        Constructs all necessary attributes for the textfile object

        Parameters
        ----------
        filename : str, optional
            name of file to read (default is None)
        initdata : list of str, optional
            can initialize the class using custom data (default is None)
        '''
        self.filename = filename
        self.data = initdata
        self.Nlines = 0
        if initdata is not None:
            self.data = initdata
            self.Nlines = len(self.data)

    @classmethod
    def readtext(self, filename):
        '''
        Creates a class instance by reading in filename to populate this object with data

        Parameters
        ----------
        filename : str
            name of file to read

        Returns
        -------
        A new textfile object
        '''
        data = self.readfile(filename)
        return textfile(filename = filename, initdata = data)

    @doesfileexist
    def readfile(filename):
        '''
        Reads a file into a list of str

        Parameters
        ----------
        filename : str
            name of file to read

        Returns
        -------
        The files data as a list of str
        '''
        f = open(filename, 'r')
        data = f.readlines()
        f.close()
        return data

    def writefile(self, writefilename = 'mytestfile.txt'):
        '''
        Writes a list of str into a file

        Parameters
        ----------
        writefilename : str, optional
            name of file to write to (default is mytestfile.txt)

        Returns
        -------
        None

        Raises
        ------
        ValueError
            If the data is not made up of a list of str
        '''
        try:
            f = open(writefilename, 'w')
            f.writelines(self.data)
            f.close()
        except:
            raise ValueError('The data you are trying to print to {:} is not made up of all strings'.format(writefilename))

    def findkey(self, keyinput, startline = 0, endline = None):
        '''
        Searchs for keyinput in textfile object and returns which lines contain keyinput

        Parameters
        ----------
        keyinput : str
            string to search for in text
        startline : int, optional
            first line in file to search, inclusive (default is 0)
        endline : int, optional
            last line in file to search, not inclusive (default is None)

        Returns
        -------
        list of lines in file where keyinput appears

        Raises
        ------
        LookupError
            If keyinput is not found in file
        '''
        #finds line where key occurs in stored input
        if endline is None:
            endline = self.Nlines
        keyinput = str(keyinput)
        lines = [i for i in range(startline, endline) if keyinput in self.data[i]]
        if not lines:
            raise LookupError('Could not find "{:}" text string in the file: {:}'.format(keyinput, self.filename))
        else:
            return lines

    def extractdata(self, line, startindex, endindex = np.nan, splitchar = None):
        '''
        Returns specific piece of data within a line of text determined by the indices and splitchar

        Parameters
        ----------
        line : int
            line of text to search
        startindex : int
            index of split text to return (or start to return)
        endindex : int, optional
            index of split text to stop returning, not inclusive (default is NaN because None means return everything after startindex)
        spitchar : str, optional
            character used to split the line of text (default is None, which really means split on any whitespace; note ' ' splits on 1 whitespace character)

        Returns
        -------
        piece of data within a line of text

        Raises
        ------
        ValueError
            If startindex is greater than the number of elements in the split line of text
            If endindex is not an integer and is greater than the number of elements in the split line of text
        '''
        if line < self.Nlines:
            dataline = self.data[line].split(splitchar)
        if startindex >= len(dataline):
            raise ValueError('startindex of {:} cannot be greater than or equal to elements in split line: "{:}"'.format(startindex, dataline))

        if endindex is not np.nan and endindex != startindex:
            try:
                return dataline[startindex:endindex]
            except:
                raise ValueError('endindex of {:} must be an integer and cannot be greater than elements in split line: "{:}"'.format(endindex, dataline))
        else:
            return dataline[startindex]

    def discarddata(self):
        '''Deletes the data in the textfile data attribute to save memory'''
        self.data = None

## Class procar: 
"PROCAR" is a vasp output file that contains information on the atomic contribution of partial density of states (PDOS)
on various bands. The goal of this 'procar' class is to extract all necessary information from the file PROCAR. See the 
text commented inside the block of codes below for more information.

In [ ]:
class procar(textfile):
    '''
    A class to reads and processes a VASP PROCAR to extract all the interesting information

    ...

    Inherits
    --------
    from textfile class

    Attributes
    ----------
    Nk : int
        number of k-points in data
    Nspin : int
        number of spin channels in data
    Nb : int
        number of bands in data
    Nions : int
        number of ions in data
    Norbs : int
        number of orbitals in data
    orbs : list of str
        list of orbitals contained in file
    kwts : list of floats
        weights of each k-point in data
    eigs : array of floats
        eigenvalues in the data
    occs : array of floats
        orbital occupations in the data
    projectors : array of floats
        projectors for the system
    projectorsum : float
        sum of all projectors in system
    Nelec : int
        total electrons in system
    sumkwts : float
        sum of all k-point weights in system
    projpdos : array of floats
        PDOS data created by assigning Gaussians to each eigenvalue and summing over energies
    E : array of floats
        energy range of projpdos
    EF : float
        Fermi level of system
    occspot : int
        index of highest E value that is below EF
    total : float
        total integral of all PDOS states below EF

    Methods
    -------
    readprocar(readfilename = 'PROCAR'):
        Creates a class instance by reading in filename to populate this object with data

    readNk():
        Reads number of k-points in PROCAR

    readNspin():
        Reads number of spins in PROCAR

    readNb():
        Reads number of bands to process in PROCAR

    readNions():
        Reads number of ions to process in PROCAR

    processfile():
        Outer loop over k-points to process PROCAR

    processkblock(isp, ik, startline, endline):
        Extracts k-point specific info and prepares projector block processing

    processbandblock(isp, ik, startline, endline):
        Collects projector data for a specific k-point and band

    getNelec():
        Checks that orbital occupations make physical sense and sum to be near an integer

    findoccspot():
        Returns index of last data point that lies below Fermi level (already shifted to 0)

    normgaussian(x, mu, sigma = 0.0048):
        Return value of Gaussian function with area 1, mean = mu, and variance = sigma, at a value = x

    calcpdosfromeigs(erange, EFermi, sigma = 0.0048):
        Create a PDOS from the data eigenvalues and projectors

    integratepdos():
        Calculate total integrated PDOS, ideally sums to Nelec

    getatomorbs(atom):
        User defined method that returns which orbital the PROCAR spd projections should be labeled as

    getatomorbfill(atomorbs):
        User defined method that returns which indices of the summary matrix should be overwritten

    writesummaryfile(myposcar, compareprocar, writefilename = 'summaryfile.txt'):
        Creates and writes the data in this object into a summary file
    '''

    def __init__(self, filename = None, initdata = None):
        '''
        Constructs all necessary attributes for the textfile object

        Parameters
        ----------
        filename : str, optional
            name of file to read (default is None)
        initdata : list of str, optional
            can initialize the class using custom data (default is None)

        Raises
        ------
        ValueError
            If try and create a procar instance without either reading in a PROCAR or providing your own data
        '''

        super().__init__(filename = filename, initdata = initdata)

        if initdata is None:
            raise ValueError('You must provide some initdata by reading in a PROCAR via readprocar')

        self.Nk = self.readNk()
        self.Nspin = self.readNspin()
        self.Nb = self.readNb()
        self.Nions = self.readNions()
        #input order of orbitals from PROCAR
        self.Norbs = 9
        self.orbs = ['s', 'py', 'pz', 'px', 'dxy', 'dyz', 'dz2', 'dxz', 'dx2y2']

        self.kwts = np.zeros([self.Nspin, self.Nk])
        self.eigs = np.zeros([self.Nspin, self.Nk, self.Nb])
        self.occs = np.zeros([self.Nspin, self.Nk, self.Nb])
        self.projectors = np.zeros([self.Nspin, self.Nk, self.Nb, self.Nions, self.Norbs])

        self.processfile()

        self.projectorssum = np.sum(self.projectors) / self.Nk / self.Nb

        self.Nelec = self.getNelec()
        self.sumkwts = np.sum(self.kwts, axis=1)

        self.projpdos = None
        self.E = None
        self.EF = None
        self.occspot = None
        self.total = None

        self.discarddata()

    @classmethod
    def readprocar(self, readfilename = 'PROCAR'):
        '''Creates a class instance by reading in filename to populate this object with data'''
        readdata = self.readfile(readfilename)
        return procar(filename = readfilename, initdata = readdata)

    def readNk(self):
        '''Reads number of k-points in PROCAR'''
        return int(self.extractdata(line = 1, startindex = 3))

    def readNspin(self):
        '''
        Reads number of spins in PROCAR

        The spin down PROCAR is just appended to spin up PROCAR, thus, it has 2x the number of k-blocks
        '''
        Nkeys = len(self.findkey('k-point '))
        return int(Nkeys / self.Nk)

    def readNb(self):
        '''Reads number of bands to process in PROCAR'''
        return int(self.extractdata(line = 1, startindex = 7))

    def readNions(self):
        '''Reads number of ions to process in PROCAR'''
        return int(self.extractdata(line = 1, startindex = 11))

    def processfile(self):
        '''Outer loop over k-points to process PROCAR'''
        kptlines = self.findkey('k-point ')
        kptlines.append(self.Nlines)
        for isp in range(self.Nspin):
            for ik in range(self.Nk):
                lineindex = isp * self.Nk + ik
                self.processkblock(isp, ik, kptlines[lineindex], kptlines[lineindex+1])

    def processkblock(self, isp, ik, startline, endline):
        '''Extracts k-point specific info and prepares projector block processing'''
        self.kwts[isp,ik] = self.extractdata(line = startline, startindex = -1)
        self.processbandblock(isp, ik, startline + 2, endline)

    def processbandblock(self, isp, ik, startline, endline):
        '''
        Collects projector data for a specific k-point and band

        Has projector data for all ions and orbitals in a block
        '''
        bandlines = self.findkey('band ', startline = startline, endline = endline)
        bandlines.append(self.Nlines)
        for ib in range(self.Nb):
            self.eigs[isp,ik,ib] = self.extractdata(line = bandlines[ib], startindex = 4)
            self.occs[isp,ik,ib] = self.extractdata(line = bandlines[ib], startindex = 7)

            projline = self.findkey('ion ', startline = bandlines[ib], endline = bandlines[ib+1])[0]
            projline += 1
            projdata = self.data[projline:projline+self.Nions]
            projdata = [x.split()[1:1+self.Norbs] for x in projdata]
            projdata = [[float(y) for y in x] for x in projdata]
            self.projectors[isp,ik,ib,:,:] = projdata

    def getNelec(self, EF = 10000):
        '''Checks that orbital occupations make physical sense and sum to be near an integer'''
        threshold = 0.01  #see warning message for caveats
        threshold = 1     #just make this very large since I already checked these BEAST jobs finished correctly

        occselect = np.where(self.eigs <= EF, self.occs, np.zeros(np.shape(self.occs)))
        Nelec = np.sum(occselect) / self.Nk
        Nelecint = int(round(Nelec, 0))
        if abs(Nelec) - Nelecint > threshold:
            warnmsg = ('The {:} file is being read and processed.\n'
                        'There is a significant fractional number of electrons in this system: {:} '
                        'if we do a strict simple total occupation sum.\n'
                        'First, try rerunning this function but while providing a Fermi level.\n'
                        'A simple occupation sum like the default may not be accurate when partial '
                        'occs are present because you really should only sum occs below EF to make '
                        'it more accurate when occupation smearing is present.\n'
                        'Then, make sure job converged!'.format(self.filename, Nelec))
            warnings.warn(warnmsg)
        else:
            return Nelecint

    def findoccspot(self):
        '''Returns index of last data point that lies below Fermi level (already shifted to 0)'''
        return [i for i in range(len(self.E)) if self.E[i] <= 0][-1]

    @staticmethod
    def normgaussian(x, mu, sigma = 0.0048):
        '''Return value of Gaussian function with area 1, mean = mu, and variance = sigma, at a value = x'''
        x = np.array(x)
        mu = np.array(mu)
        return 1.0 / (sigma * (2.0 * np.pi)**0.5) * np.exp(-1.0 * (x - mu)**2 / (2.0 * sigma**2))

    def calcpdosfromeigs(self, erange, EFermi, sigma = 0.0048):
        '''
        Create a PDOS from the data eigenvalues and projectors

        Assigns gaussian to each eigenvalue and sums up contributions across the energy range of interest
        Note: sigma = 0.0048 was found empirically to provide best match between integral of DOSCAR PDOS and projector PDOS below EF

        Parameters
        ----------
        erange : array of floats
            energy range over which to calculate PDOS data, can take from DOSCAR object if want or make your own
        EFermi : float
            Fermi level of system, can take from DOSCAR object or specify your own
        sigma : float, optional
            broadening of Gaussians used to generate PDOS, larger = smoother (default = 0.0048)

        Returns
        -------
        None
        '''
        self.E = np.array(erange).copy()   #ensure there's no referencing weirdness if ever change self.E, EF is not affected by this issue
        self.occspot = self.findoccspot()
        self.EF = EFermi           #need to shift eigs by EF because erange from doscar already shifted by EF
        nedos = np.shape(self.E)[0]
        self.nedos = nedos
        self.total = np.zeros([self.Nspin, self.nedos])
        self.projpdos = np.zeros([self.Nspin, self.Nions, self.Norbs, self.nedos])

    #Notes: all of the below 3 loop blocks are equivilant. The middle one is easily fastest
        #the top one is the most readable and shows the explicit addition of many data vectors after the gaussian function
        #the middle one removes one of the loops by evaulating the gaussian function for all band eigenvalue (different gaussian mus) at once
        #   the different numpy transposes are so that the correct matrix dimension evaluations are preserved
        #the bottom loop block attempts to remove another loop by absorbing the k-point index, yet is mysteriously much slower than either of the first 2
    #     for isp in range(self.Nspin):
    #         for ik in range(self.Nk):
    #             for ib in range(self.Nb):
    #                 temp = (2.0 / self.Nspin) * self.normgaussian(self.E, self.eigs[isp, ik, ib] - self.EF, sigma) * self.kwts[isp, ik]
    #                 self.total += temp
    #                 for ii in range(self.Nions):
    #                     for io in range(self.Norbs):
    #                         self.projpdos[isp, ii, io, :] += temp * self.projectors[isp, ik, ib, ii, io]

        for isp in range(self.Nspin):
            for ik in range(self.Nk):
                ibeigs = np.array([self.eigs[isp, ik, :]]).T
                temp = (2.0 / self.Nspin) * self.normgaussian(np.tile(self.E, [self.Nb, 1]), ibeigs - self.EF, sigma) * self.kwts[isp, ik]
                self.total += np.sum(temp, axis = 0)
                for ii in range(self.Nions):
                    for io in range(self.Norbs):
                        self.projpdos[isp, ii, io, :] += np.sum(temp * np.array([self.projectors[isp, ik, :, ii, io]]).T, axis = 0)

        # for isp in range(self.Nspin):
        #     ikibeigs = np.array([self.eigs[isp, :, :]])
        #     temp = (2.0 / self.Nspin) * self.normgaussian(np.tile(self.E, [self.Nb, self.Nk, 1]).T, ikibeigs - self.EF, sigma) * np.array([self.kwts[isp, :]]).T
        #     self.total += np.sum(temp, axis = (1, 2))
        #     for ii in range(self.Nions):
        #         for io in range(self.Norbs):
        #             self.projpdos[isp, ii, io, :] += np.sum(temp * np.array([self.projectors[isp, :, :, ii, io]]), axis = (1, 2))


    def integratepdos(self):
        '''Calculate total integrated PDOS, ideally sums to Nelec'''
        if self.E is None or self.projpdos is None:
            raise ValueError('Must provide energy list and pdos to calculatepdos() first!')
        total = 0
        for isp in range(self.Nspin):
            for ii in range(self.Nions):
                for io in range(self.Norbs):
                    total += np.trapz(self.projpdos[isp, ii, io, :self.occspot], self.E[:self.occspot])
        return total

    @staticmethod
    def getatomorbs(atom):
        '''User defined method that returns which orbital the PROCAR spd projections should be labeled as'''
        if atom in ['H','He']:
            return ['1s']
        elif atom in ['Li','Be']:
            return ['2s','2p']
        elif atom in ['B','C','N','O','F','Ne']:
            return ['2s','2p']
        elif atom in ['Na','Mg']:
            return ['3s','3p']
        elif atom in ['Al','Si','P','S','Cl','Ar']:
            return ['3s','3p']
        elif atom  in ['K','Ca']:
            return ['4s','4p']
        elif atom in ['Sc','Ti','V','Cr','Mn','Fe','Co','Ni','Cu','Zn']:
            return ['4s','3d','4p']
        elif atom in ['Ga','Ge','As','Se','Br','Kr']:
            return ['4s','4p']
        elif atom in ['Rb','Sr']:
            return ['5s','5p']
        elif atom in ['Y','Zr','Zb','Mo','Tc','Ru','Rh','Pd','Ag','Cd']:
            return ['5s','4d','5p']
        elif atom in ['In','Sn','Sb','Te','I','Xe']:
            return ['5s','5p']
        elif atom in ['Cs','Ba']:
            return ['6s','6p']
        elif atom in ['Lu','Hf','Ta','W','Re','Os','Ir','Pt','Au','Hg']:
            return ['6s','5d','6p']
        elif atom in ['Tl','Pb','Bi','Po','At','Rn']:
            return ['6s','6p']
        else:
            print('need to add this atom orbitals!')
            sys.exit()

    @staticmethod
    def getatomorbfill(atomorbs):
        '''User defined method that returns which indices of the summary matrix should be overwritten'''
        allorbs = '1s 2s 2p 3s 3p 3d 4s 4p 4d 5s 5p 5d 6s 6p'
        allorbs = allorbs.split()
        order = ['s','p','d']
        keep = []
        for io,o in enumerate(allorbs):
            if o in atomorbs:
                for jj,j in enumerate(order):
                    if j in o:
                        keep.append([io,jj])
        return keep

    def writesummaryfile(self, myposcar, compareprocar, writefilename = 'summaryfile.txt'):
        '''
        Creates and writes the data in this object into a summary file

        Parameters
        ----------
            myposcar : poscar object
                poscar object used to determine loop indices and how to group the projectors by ion type
            compareprocar : procar object
                procar object used to compare how the eigenvalues for the same (atom, spin, k-point, band, orbital) indices have shifted
            writefilename : str, optional
                name of file to write to (default is 'summaryfile.txt')

        Returns
        -------
        None

        Raises
        ------
        ValueError
            If do not input a poscar object
            If do not input a procar object
            If the number of spins in both this procar object and inputted one do not match
            If the number of k-points in both this procar object and inputted one do not match
        '''
        if myposcar is None:
            raise ValueError('Must input a poscar() object for atomic type information')
        if compareprocar is None:
            raise ValueError('Must input a procar() object to compute eigenvalue shifts (new_procar - this_procar).\nCould input this object for all shifts to be 0')

        if self.Nspin != compareprocar.Nspin:
            raise ValueError('The number of spins in the procar objects do not match! self has {:} while compareprocar has {:} spins'.format(self.Nspin, compareprocar.Nspin))
        if self.Nk != compareprocar.Nk:
            raise ValueError('The number of k-points in the procar objects do not match! self has {:} while compareprocar has {:} k-points'.format(self.Nk, compareprocar.Nk))

        #the number of bands between vasp jobs can be slightly different if different
        #NPAR or Nprocessors were used; this param ensures do not try to write outside of available array sizes
        minbands = min([self.Nb, compareprocar.Nb])

        sumfile = textfile()

        orbs = '1s 2s 2p 3s 3p 3d 4s 4p 4d 5s 5p 5d 6s 6p'
        data = ['index atomtype isp ik ib ' + orbs + ' sum occPBE occHSE06 occdiff pbe_eig eigshift(eV)\n']
        orbs = orbs.split()

        #create arrays to hold summed projector data
        projinit = self.projectors
        proj = np.zeros([myposcar.Nattype, self.Nspin, self.Nk, self.Nb, len(orbs)])
        #loop over ions in system
        for ii in range(self.Nions):
            #get atom types and which valence orbitals the projector refer to
            atom = myposcar.atomnamelist[ii]
            atomorbs = self.getatomorbs(atom)
            atomorbfill = self.getatomorbfill(atomorbs)
            attype = myposcar.atomtypelist[ii]

            #sum up the projector data by the m quantum number to combine for example px, py, pz into p
            #   projector order is self.orbs
            projinit = self.projectors[:, :, :, ii, :]
            projtemp = np.zeros([myposcar.Nattype, self.Nspin, self.Nk, self.Nb, 3])
            projtemp[attype, :, :, :, 0] = projinit[:, :, :, 0]
            projtemp[attype, :, :, :, 1] = np.sum(projinit[:, :, :, 1:4], axis = 3)
            projtemp[attype, :, :, :, 2] = np.sum(projinit[:, :, :, 4:], axis = 3)

            for ix, x in enumerate(atomorbfill):
                proj[attype, :, :, :, x[0]] = projtemp[attype, :, :, :, x[1]]

        #loop over all indices to collect indices, summed projector data, and eigenvalue shifts
        c = 0
        for ii in range(myposcar.Nattype):
            for isp in range(self.Nspin):
                for ik in range(self.Nk):
                    for ib in range(minbands):  #only write up to the smaller number of bands
                        projline = proj[ii, isp, ik, ib, :]
                        sumproj = sum(projline)
                        projline = np.append(projline, sumproj)
                        projline = ['{:>6.4f}'.format(x) for x in projline]
                        eigdiff = compareprocar.eigs[isp, ik, ib] - self.eigs[isp, ik, ib]
                        occdiff = compareprocar.occs[isp, ik, ib] - self.occs[isp, ik, ib]
                        eig=self.eigs[isp,ik,ib]
                        #if abs(occdiff) > 0.00001 and ii == 0:
                        #    print(isp, ik, ib, occdiff)
                        projline = '{:>6} {:>2} {:>2} {:>4} {:>4} '.format(c,myposcar.atoms[ii],isp,ik,ib) + \
                                ' '.join(projline) + \
                                ' {:>11.8f} {:>11.8f} {:>11.8}'.format(self.occs[isp, ik, ib], compareprocar.occs[isp, ik, ib], occdiff) + ' {:>-13.8f}'.format(eig) + \
                                ' {:>-13.8f}'.format(eigdiff) + '\n'
                        data.append(projline)
                        c += 1

        sumfile.data = data
        sumfile.writefile(writefilename)



## class vector : 
This class controls the length of the vector relevent for input. Find more information on the text inside the following
block of codes

In [ ]:
class vector():
    '''
    Class for representing n length vector, initialized by list of values [1, 2, 3, ...]

    ...

    Attributes
    ----------
    v : array of floats
        array containing elements of vector
    norm : float
        the norm of the vector

    Methods
    -------
    getnorm():
        Calculate norm (i.e. the length) of the vector

    anglewithvector(v2):
        Calculate angle of the vector with another vector v2
    '''

    def __init__(self, v):
        self.v = np.array(v)
        self.norm = self.getnorm()

    def __array__(self, dtype=None):
        '''so vector class object behaves like normal array'''
        if dtype:
            return np.array(self.v, dtype=dtype)
        else:
            return np.array(self.v)

    def getnorm(self):
        '''Calculate norm (i.e. the length) of the vector'''
        return np.linalg.norm(self.v)

    def anglewithvector(self, v2):
        '''
        Calculate angle of the vector with another vector v2

        Clip method prevents arccos from returning nan for edge cases
        '''
        return np.degrees(np.arccos(np.clip(np.dot(self.v, v2) / self.norm / np.linalg.norm(v2), -1.0, 1.0)))

## Class poscar: 
'POSCAR' is a input file for VASP calculations that contains information on the geometry of the atom/molecules/compound
used in the calculation. The class 'poscar' helps extract all necessary information from the file 'POSCAR'. Details can be 
found by commented text inside the following block of codes.

In [ ]:
class poscar(textfile):
    '''
    A class to reads and processes a VASP PROCAR to extract all the interesting information and/or modify structure

    ...

    Inherits
    --------
    from textfile class

    Attributes
    ----------
    scalefactor : float
        factor that scales the lattice matrix by a constant multiplicative factor (default is 1.0)
    lattice : 3x3 array of floats
        lattice matrix of structure
    avec : array of 3 floats
        a vector of structure
    bvec : array of 3 floats
        b vector of structure
    cvec : array of 3 floats
        c vector of structure
    a, b, c, alpha, beta, gamma : float
        length of a, b, c vectors and angle between b/c, a/c, and a/b vectors, respectively
    volume : float
        volume of lattice matrix
    atoms : array of str
        list of atom species in structure
    atomnums : array of ints
        list of numbers of atom species in structure in same order as atoms
    atomdict : dict of atoms and atomnums
        combined atoms and atomnums information
    Nattype : int
        number of different atom types in structure
    Nat : int
        number of total atoms in the structure
    atomtypelist : list of ints
        list of elements in structure of length Nat, where each entry is an atom index, starting at 0
    atomnamelist : list of str
        same as atomtypelist but uses element symbols instead of an atom index
    formula : str
        chemical formula of structure based on atoms and atomnums
    hasSD : bool
        flag that logs if structure has selective dynamics or not
    origcoordstype : str
        log of the original coordinates of the structure
    rawcoords : array of floats
        array containing the raw coordinates of atoms as directly read from file
    sd : array of bool
        array containing bool flags on (x, y, z) status of each atom's selective dynamics
    dircoords : array of floats
        array containing the coordinates of atoms in structure in direct coordinates
    cartcoords: array of floats
        array containing the coordinates of atoms in structure in cartesian coordinates

    Methods
    -------
    readposcar(readfilename = 'POSCAR'):
        Creates a class instance by reading in filename to populate this object with data

    writeposcar(writefilename = 'POSCAR', overwritefile = True,
                    scalefactor = 1.0, coordstype = 'direct', noSD = False):
        Writes poscar object to a file in POSCAR format

    readscalefactor():
        Returns the POSCAR lattice scale factor

    readlattice():
        Returns the structures lattice matrix

    scalelattice(scalefactor):
        Scales the lattice matrix by the scalefactor

    getvolume():
        Calculates the volume of the lattice matrix

    readatoms():
        Return list of atom species in the structure

    readatomnums():
        Return list of number of atom species in the structure

    makeatomdict():
        Returns a dict of atom type and number for the structure

    makeatomtypelist():
        Return list of atom types of length Nat, where atom types start with 0 index

    makeformula():
        Return chemical formula for structure

    readSD():
        Determine if a POSCAR uses selective dynamics

    readcoordstype():
        Results type of atomic coordinates used in structure

    readcoords():
        Read atomic coordinates from POSCAR

    cart2dir(coordstoconvert = None):
        Convert cartesian coordinates to fractional relative to lattice matrix

    dir2cart(coordstoconvert = None):
        Convert fractional coordinates to direct relative to lattice matrix

    addatomsymbol(symbol):
        Adds an element symbol to the structure

    translatepoint(point):
        Translate coordinate point(s) so that all values are between 0 and 1

    addatom(symbol, atcoords, atcoordstype = 'direct',
                atSD = [False, False, False], translatecoords = True):
        Add an element to the structure and updates the atom type lists

    freezeatoms(freezedir = 'c', lowcoord = 0, highcoord = 1,
                coordstype = 'direct', overwritesd = False):
        Add selective dynamics = T (True) for atoms with coordinates along freezedir that fall within lowcoord and highcoord (inclusive)
    '''

    def __init__(self, filename = None, initdata = None, absorbscalefactor = True):
        '''
        Constructs all necessary attributes for the textfile object

        Parameters
        ----------
        filename : str, optional
            name of file to read (default is None)
        initdata : list of str, optional
            can initialize the class using custom data (default is None)
        absorbscalefactor : bool
            if True, multiply the lattice matrix by the POSCAR scalefactor

        Raises
        ------
        ValueError
            If try and create a poscar instance without either reading in a POSCAR or providing your own data
        '''

        super().__init__(filename = filename, initdata = initdata)

        if initdata is None:
            raise ValueError('You must provide some initdata by reading in a POSCAR via readposcar')

        self.scalefactor = self.readscalefactor()
        self.lattice = self.readlattice()
        if absorbscalefactor:
            self.scalelattice(self.scalefactor)
            self.scalefactor = 1.0
        self.avec, self.bvec, self.cvec = vector(self.lattice[0]), vector(self.lattice[1]), vector(self.lattice[2])
        self.a = self.avec.norm
        self.b = self.bvec.norm
        self.c = self.cvec.norm
        self.alpha = self.bvec.anglewithvector(self.cvec)
        self.beta = self.avec.anglewithvector(self.cvec)
        self.gamma = self.avec.anglewithvector(self.bvec)
        self.volume = self.getvolume()

        self.atoms = self.readatoms()
        self.atomnums = self.readatomnums()
        self.atomdict = self.makeatomdict()
        self.Nattype = len(self.atoms)
        self.Nat = sum(self.atomnums)
        self.atomtypelist, self.atomnamelist = self.makeatomtypelist()
        self.formula = self.makeformula()

        self.hasSD = self.readSD()
        self.origcoordstype = self.readcoordstype()

        self.rawcoords, self.sd = self.readcoords()
        if self.origcoordstype == 'direct':
            self.dircoords = self.translatepoint(self.rawcoords)
            self.cartcoords = self.dir2cart()
        elif self.origcoordstype == 'cartesian':
            self.cartcoords = self.rawcoords
            self.dircoords = self.cart2dir()

        self.discarddata()

    @classmethod
    def readposcar(self, readfilename = 'POSCAR'):
        '''Creates a class instance by reading in filename to populate this object with data'''
        readdata = self.readfile(readfilename)
        return poscar(filename = readfilename, initdata = readdata)

    def writeposcar(self, writefilename = 'POSCAR', overwritefile = True,
                    scalefactor = 1.0, coordstype = 'direct', noSD = False):
        '''
        Writes poscar object to a file in POSCAR format

        Parameters
        ----------
        writefilename : string, optional
            name of file to write (default is 'POSCAR')
        overwritefile : bool, optional
            if True overwrite exising file of the same name (default is True)
        scalefactor : float, optional
            factor that scales the lattice matrix by a constant multiplicative factor (default is 1.0)
        coordstype : str, optional, choices : ['direct', 'cartesian']
            type of coodinate system for ions (default is 'direct')
        noSD : bool, optional
            if True, do not write selective dynamics flags (default is False)

        Returns
        -------
        None

        Raises
        ------
        ValueError
            If do not provide coordinates in direct or cartesian systems
        '''

        while (not overwritefile and os.path.isfile(writefilename)):
            suffix = '_temp'
            print('Updated {:} to {:} because {:} already exists!'.format(writefilename, writefilename + suffix, writefilename))
            writefilename += suffix
        if coordstype.lower() not in ['direct', 'cartesian']:
            raise ValueError('coordstype must be direct or cartesian!')
        if scalefactor != 1.0:
            self.scalefactor = scalefactor

        self.data = [self.formula + '\n',
                    str(self.scalefactor) + '\n']
        self.data.extend([stringifyfloatarray(x) + '\n' for x in self.lattice])
        self.data.append(stringifyarray(self.atoms, minspaces = 2) + '\n')
        self.data.append(stringifyarray(self.atomnums, minspaces = 2) + '\n')

        if not noSD:
            self.data.append('Selective Dynamics\n')
        if coordstype == 'direct':
            coordstype = 'Direct'
            atcoords = [stringifyfloatarray(x) for x in self.dircoords]
        if coordstype == 'cartesian':
            coordstype = 'Cartesian'
            atcoords = [stringifyfloatarray(x) for x in self.cartcoords]
        if not noSD:
            convertedsd = np.where(self.sd == True, 'T', 'F')
            atcoords = [x + ' ' + stringifyarray(convertedsd[xx]) + '\n' for xx, x in enumerate(atcoords)]
        else:
            atcoords = [x + '\n' for x in atcoords]

        self.data.append(coordstype + '\n')
        self.data.extend(atcoords)
        self.data.append('\n')

        self.writefile(writefilename)

    def readscalefactor(self):
        '''Returns the POSCAR lattice scale factor'''
        return float(self.extractdata(line = 1, startindex = 0))

    def readlattice(self):
        '''Returns the structures lattice matrix'''
        temp = self.data[2:5]
        temp = [[float(y) for y in x.split()] for x in temp]
        return np.array(temp)

    def scalelattice(self, scalefactor):
        '''Scales the lattice matrix by the scalefactor'''
        self.lattice *= scalefactor

    def getvolume(self):
        '''Calculates the volume of the lattice matrix'''
        return np.dot(np.cross(self.avec, self.bvec), self.cvec)

    def readatoms(self):
        '''Return list of atom species in the structure'''
        line = 5
        return self.data[line].split()

    def readatomnums(self):
        '''Return list of number of atom species in the structure'''
        line = 6
        return [int(x) for x in self.data[line].split()]

    def makeatomdict(self):
        '''Returns a dict of atom type and number for the structure'''
        return dict(zip(self.atoms, self.atomnums))

    def makeatomtypelist(self):
        '''Return list of atom types of length Nat, where atom types start with 0 index'''
        tempi = []
        tempn = []
        for iat in range(self.Nattype):
            for ia in range(self.atomnums[iat]):
                tempi.append(iat)
                tempn.append(self.atoms[iat])
        return tempi, tempn

    def makeformula(self):
        '''Return chemical formula for structure'''
        return ''.join([''.join([str(y) for y in x]) for x in zip(self.atoms, self.atomnums)])

    def readSD(self):
        '''Determine if a POSCAR uses selective dynamics'''
        line = 7
        temp = self.extractdata(line = line, startindex = 0)
        if temp[0].lower() == 's':
            return True
        else:
            return False

    def readcoordstype(self):
        '''Results type of atomic coordinates used in structure'''
        if self.hasSD:
            line = 8     #this line is pushed down by one if selective dynamics line present
        else:
            line = 7
        temp = self.extractdata(line = line, startindex = 0)
        if temp[0].lower() == 'k' or temp[0].lower() == 'c':
            return 'cartesian'
        else:
            return 'direct'

    def readcoords(self):
        '''Read atomic coordinates from POSCAR'''
        startline = 8
        if self.hasSD:
            startline = 9
        temp = self.data[startline:startline+self.Nat]
        temp = [x.split() for x in temp]
        coords = [x[:3] for x in temp]
        if self.hasSD:
            sd = [x[3:6] for x in temp]
        else:
            sd = [[False, False, False] for i in range(self.Nat)]
        return np.array([[float(y) for y in x] for x in coords]), np.array(sd)

    def cart2dir(self, coordstoconvert = None):
        '''Convert cartesian coordinates to fractional relative to lattice matrix'''
        lattice = np.linalg.inv(self.lattice)
        if coordstoconvert is None:
            #if just doing an internal conversion
            return self.translatepoint(np.matmul(self.cartcoords, lattice))
        else:
            #to convert new point(s)
            return self.translatepoint(np.matmul(np.array(coordstoconvert), lattice))

    def dir2cart(self, coordstoconvert = None):
        '''Convert fractional coordinates to direct relative to lattice matrix'''
        if coordstoconvert is None:
            #if just doing an internal conversion
            return self.translatepoint(np.matmul(self.dircoords, self.lattice))
        else:
            #to convert new point(s)
            return self.translatepoint(np.matmul(np.array(coordstoconvert), self.lattice))

    def addatomsymbol(self, symbol):
        '''Adds an element symbol to the structure'''
        if not symbol in self.atoms:
            self.atoms.append(symbol)
            self.atomnums.append(1)
        else:
            self.atomdict[symbol] += 1
            self.atomnums = [self.atomnums[i]+1 if symbol == self.atoms[i] else self.atomnums[i] for i in range(self.Nattype)]

    @staticmethod
    def translatepoint(point):
        '''
        Translate coordinate point(s) so that all values are between 0 and 1

        Doing this just makes poscar direct coordinates cleaner and possibly
        easier to visualize if later use something like Jmol to visualize, where
        the structure is displayed in cartesian coordinates automatically
        '''
        if hasattr(point, '__len__'):   #return array/lists for inputted array/list
            return np.array(point % 1)
        else:
            return point % 1   #return a scalar otherwise; will error for str

    def addatom(self, symbol, atcoords, atcoordstype = 'direct',
                atSD = [False, False, False], translatecoords = True):
        '''
        Add an element to the structure and updates the atom type lists

        Parameters
        ----------
        symbol : str
            elemental symbol
        atcoords : list of 3 floats
            (x, y, z) coordinates of the atom
        atcoordstype : string, optional, choices : ['direct', 'cartesian']
            coordinate system of atcoords, determines which coordinate transformations are done (default is 'direct')
        atSD : list of 3 bools, optional
            (x, y, z) selective dynamics of the atom (default is [False, False, False])
        translatecoords : bool, optional
            if True, translate the atcoords point so that all values are between 0 and 1 (default is True)

        Returns
        -------
        None

        Raises
        ------
        ValueError
            If do not provide 3 coordinates for the atom (x, y, z)
        '''
        if len(atcoords) != 3 or len(atSD) != 3:
            raise ValueError('Must input arrays with exactly 3 elements like: [0.5, 0.4, 0.6]')

        atcoords = np.array(atcoords).copy()

        self.addatomsymbol(symbol)
        self.atomdict = self.makeatomdict()
        self.Nattype = len(self.atoms)
        self.Nat += 1
        self.atomtypelist, self.atomnamelist = self.makeatomtypelist()
        self.formula = self.makeformula()

        if translatecoords:
            atcoords = self.translatepoint(atcoords)
        if atcoordstype == 'cartesian':
            self.cartcoords = np.append(self.cartcoords, [atcoords], axis = 0)
            atcoords = self.cart2dir(np.array(atcoords))
            self.dircoords = np.append(self.dircoords, [atcoords], axis = 0)
        elif atcoordstype == 'direct':
            self.dircoords = np.append(self.dircoords, [atcoords], axis = 0)
            atcoords = self.dir2cart(np.array(atcoords))
            self.cartcoords = np.append(self.cartcoords, [atcoords], axis = 0)
        self.sd = np.append(self.sd, [atSD], axis = 0)  #np.append assigns new object in memory so will
                                             #not get referencing issues if atSD input changes outside function

    def freezeatoms(self, freezedir = 'c', lowcoord = 0, highcoord = 1,
                    coordstype = 'direct', overwritesd = False):
        '''
        Add selective dynamics = T (True) for atoms with coordinates along freezedir that fall within lowcoord and highcoord (inclusive)

        Set overwritesd to True to fix any mistakes
        '''
        if coordstype.lower() == 'direct':
            lowcoord = self.translatepoint(lowcoord)
            highcoord = self.translatepoint(highcoord)
            #transpose because getting list of bools for a direction and will
            #convert back to keep in a list per atom
            atcoords = self.dircoords.T
        else:
            #transpose because getting list of bools for a direction and will
            #convert back to keep in a list per atom
            atcoords = self.cartcoords.T
        alldirs = ['a','b','c']

        newsd = [(atcoords[xx, :] >= lowcoord) & (atcoords[xx, :] <= highcoord)
                 if x in freezedir
                 else [False]*self.Nat
                 for xx,x in enumerate(alldirs)]
        newsd = np.array(newsd).T   #get bools back to column format

        if overwritesd:
            self.sd = newsd
        else:
            self.sd = self.sd | newsd  #only True if original or new SD have a True for a direction
        if np.any(newsd):
            self.hasSD = True
        if not np.any(self.sd):   #if all SD set to False
            self.hasSD = False


## Process the PROCAR files from all directories
We now have all necessary output files form vasp calculations and the tools to extract the information necessary for training
the ML model. The compressed tar.gz file 'beast_calculations.tar.gz' contain 110 folders each for 110 compounds utilized in
our calculation. Each of those folder contain a subfolder 'dftredo' which was done in order to get the files containing 
information on PBE charge transfer. Actually, these files were overwritten by HSE06 calculations earlier. Therefore we needed
it to be redone. We read PROCAR corresponding to PBE from this folder.
Following block of code processes all the procar files from all the folders (110 compounds) in our calculations. 
The final result is the generation of the file 'projectorsummary.txt' inside each of those 110 folders. The last column of 
this file contains the difference between HSE06 and PBE eigen values for various band and k-point index. While other columns
before it display the atomic-orbital contribution on this difference. 

In [ ]:
#dirs = 'Ag2O1 Al1As1 Al1N1 Al1N1-rs Al1N1-zb Al1P1 Al1Sb1 Ba1O1 Ba1Se1 Ba2Sn1O4 Bi2Se3 Ca1Cl2-22904 Ca1O1-bn Ca1O1-na Ca1O1-rs Ca1O1-zb Ca1S1-bn Ca1S1-na Ca1S1-rs Ca1S1-zb Ca1Se1 Ca1Se1-bn Ca1Se1-na Ca1Se1-zb Ca1Ti1O3-4019 Cd1F2 Cd1O1-bn Cd1O1-na Cd1O1-rs Cd1O1-wz Cd1O1-zb Cd1S1-na Cd1S1-rs Cd1S1-wz Cd1S1-zb Cd1Se1-na Cd1Se1-wz Cd1Se1-zb Cd1Te1 Co1O1-wz Co1O1-zb Cs2Ag1Bi1Br6-1078250 Cu1Cl1-1184046 Cu1Cl1-1225654 Cu1Cl1-22914 Cu1Cr1O2 Cu1Mn1O2 Cu2O1 Fe1O1-1279742 Fe2O3-19770 Ga1N1 Ga1N1-rs Ga1N1-zb Ga1P1 Ge1O2 Ge1S1 Ge1Se1 In1N1 In1P1 In2Se3 K1Cl1-23193 Li1Cr1O2 Li1F1-1138 Li2O1 Li3N1 Mg1O1-bn Mg1O1-na Mg1O1-rs Mg1O1-zb Mg1S1-na Mg1S1-rs Mg1S1-wz Mg1S1-zb Mg1Se1-na Mg1Se1-rs Mg1Se1-wz Mg1Se1-zb Mg1Te1-bn Mg1Te1-na Mg1Te1-rs Mg1Te1-wz Mg1Te1-zb Mn1O1 Mn1O1-wz Mn1O1-zb Mn1O2 Mn1S1 Mn1S1-na Mn1S1-wzaf1 Mn1S1-zb Mn1Se1 Mn1Se1-na Mn1Se1-wzaf1 Mn1Se1-zb Mo1S2-2815 Mo1Se2-1027692 Na1Cl1-22862 Na2O1 Ni1O1 Pb1S1 Pb1Se1 Pb1Te1 Si1-149 Sn1Te1 Sr1Ti1O3-5229 Ti1Pb1O3-20459 W1O3-19443 W1S2-9813 Zn1O1-1986 Zn1O1-2133'
#dirs='Ag2O1'

#os.chdir('jacob_modules/')
cwd=os.getcwd()
print(cwd)
if 'beast_calculations' not in os.listdir(cwd):
    import tarfile
    file=tarfile.open('beast_calculations.tar.gz')
    file.extractall()
    file.close()
folders=os.path.join(cwd,'beast_calculations/')
dirs=os.listdir(folders)
#dirs = dirs.split()
#print(dirs)
#print(folders)
#dirs=['Ag2O1']
for dd,d in enumerate(dirs):
    print(d)
    os.chdir(folders+d)

    #read files
    #read procars
    procarpbe = procar.readprocar('dftredo/PROCAR') # uncomment it to read pdos contribution from pbe
    procarhse = procar.readprocar('PROCAR') # uncomment this line to read pdos contribution from hse
    #print(procarpbe)
    #print("hse_below\n\n\n")
    #print(procarhse)
    #read poscar
    poscarorig = poscar.readposcar('POSCAR') 

    #write summary file
    procarpbe.writesummaryfile(poscarorig, procarhse, writefilename = 'projectorsummary.txt') #uncomment this line for summary
    #procarpbe.writesummaryfile(poscarorig, procarpbe, writefilename = str(d)+'_projectorsummary.txt')
    #example of how to generate a PDOS from a procar object
    #erange = np.arange(-20,20+0.0005,0.001)
    #procarpbe.calcpdosfromeigs(erange, <the Fermi level for each system as a float>, sigma = 0.0048)
    #procarpbe.projpdos contains the pdos information in an array of size (Nspin, Nions, Norbs = 9, Nedos = len(erange))

    os.chdir('../')

    #sys.exit()



## ML part: